In [208]:
import chainlit as cl
import os
from llama_index.core import Settings, VectorStoreIndex, StorageContext, SimpleDirectoryReader, Document
from llama_index.llms.together import TogetherLLM
from llama_index.embeddings.together import TogetherEmbedding
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core import PromptTemplate

llm = TogetherLLM(
    model="meta-llama/Meta-Llama-3-70B-Instruct-Turbo", api_key="dae9b33beef8cc57bc9475d53519e9e70037e6820b8c2d54b0b2f349e11101b0"
)

embed_model = TogetherEmbedding(
    model_name = "togethercomputer/m2-bert-80M-8k-retrieval", api_key="dae9b33beef8cc57bc9475d53519e9e70037e6820b8c2d54b0b2f349e11101b0"
)

In [209]:
documents = SimpleDirectoryReader(input_dir="./data", recursive=True).load_data()
vector_store = MilvusVectorStore(uri="./milvus_demo_2.db", dim=768, overwrite=True)

2024-07-20 21:27:13 - Created new connection using: 64fbdc8740924a8c87af6eece48211a5


2024-07-20 21:27:13 - Successfully created collection: llamacollection
2024-07-20 21:27:14 - Successfully created an index on collection: llamacollection


In [210]:
# local_model_endpoint = "http://localhost:8000"
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(embed_model=embed_model,documents=documents,storage_context=storage_context)

In [211]:
Settings.embed_model = embed_model
Settings.llm = llm

In [212]:

index = VectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()
result = query_engine.query("Can you give me some names of common fever medications?")

2024-07-20 21:27:19 - HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"


In [213]:
print(result)

Acetaminophen, ibuprofen, and aspirin are some common fever medications.


In [222]:
promptTemplate = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
    "If the question is out of the scope of the given information, answer to the best of your ability"
)

In [223]:
qa_template = PromptTemplate(promptTemplate)

In [224]:
question = "What medications should I take for a urinary tract information?"
prompt = qa_template.format(context_str = documents, query_str = question)
response = llm.complete(prompt)
print(response.text)

2024-07-20 21:29:55 - HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
Based on the provided information, the answer to the question "What medications should I take for a urinary tract infection?" is that acetaminophen or ibuprofen can help relieve the pain associated with urinary tract infections. However, it's important to note that these medications do not cure infections, and you should see a doctor for proper treatment.


In [226]:
question = "Who is Donald Trump?"
prompt = qa_template.format(context_str = documents, query_str = question)
response = llm.complete(prompt)
print(response.text)

2024-07-20 22:22:27 - HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
The provided context information appears to be about two documents containing text about aspirin and tylenol, respectively. There is no mention of Donald Trump in the provided information.

To the best of my ability, I can tell you that Donald Trump is the 45th President of the United States, serving from 2017 to 2021. However, this information is not related to the provided context about aspirin and tylenol.
